# Make an interactive map of temperature in the PNW using Folium

In [8]:
import pandas as pd
import numpy as np
import os
import panel as pn
import xarray as xr
import folium as fm
import matplotlib
import matplotlib.pyplot as plt
import hvplot.pandas
import branca

### Load sample data

In [2]:
# define data file name
data_path = '/Users/raineyaberle/Courses/GEOS_505_ResearchComputing/data/CFS_Sample_Series.nc'

# load data as xarray.Dataset
data = xr.open_dataset(data_path)
data

<xarray.Dataset>
Dimensions:     (time: 5, latitude: 190, longitude: 384)
Coordinates:
  * time        (time) datetime64[ns] 2018-10-31 2018-10-31 ... 2018-10-31
    step        (time) timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 89.28 88.34 87.4 ... -87.4 -88.34 -89.28
  * longitude   (longitude) float64 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
    valid_time  (time) datetime64[ns] ...
Data variables: (12/53)
    uflx        (time, latitude, longitude) float32 ...
    vflx        (time, latitude, longitude) float32 ...
    shtfl       (time, latitude, longitude) float32 ...
    lhtfl       (time, latitude, longitude) float32 ...
    t           (time, latitude, longitude) float32 ...
    sdwe        (time, latitude, longitude) float32 ...
    ...          ...
    snohf       (time, latitude, longitude) float32 ...
    vbdsf       (time, latitude, longitude) float32 ...
    vddsf       (time, latitude, longitude) float32 ...
    nbdsf       (time, latitude, longitude) float32 ...
    nddsf       (time, latitude, longitude) float32 ...
    srweq       (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2022-11-14T11:21 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
data.valid_time.data

array(['2018-10-31T00:00:00.000000000', '2018-10-31T06:00:00.000000000',
       '2018-10-31T12:00:00.000000000', '2018-10-31T18:00:00.000000000',
       '2018-11-01T00:00:00.000000000'], dtype='datetime64[ns]')

### Subset the data in space and time

In [3]:
# grab a spatial subset for the first time slice
data_PNW = data.where((data.latitude > 35) & (data.latitude < 50)
                    & (data.longitude > 230) & (data.longitude < 250), drop=True)
data_PNW_ti = data_PNW.isel(time=0)

In [ ]:
data_PNW

### Create interactive map of the subset data

In [ ]:
# create map
m = fm.Map(location=[np.nanmean(data_PNW_ti.latitude.data),  # mean latitude value in data 
                     np.nanmean(data_PNW_ti.longitude.data)], # mean longitude value in data
           zoom_start=4, # map zoom to start
           tiles='StamenTerrain', # basemap
           width=500, # map width
           height=400) # map height

# get image extent (must specify image bounds when plotting)
xmin, xmax = np.min(data_PNW_ti.longitude.data), np.max(data_PNW_ti.longitude.data)
ymin, ymax = np.min(data_PNW_ti.latitude.data), np.max(data_PNW_ti.latitude.data)

# create colormap (from dark orange --> white --> blue)
top = matplotlib.cm.get_cmap('Oranges_r', 128)
bottom = matplotlib.cm.get_cmap('Blues', 128)
newcolors = np.vstack((top(np.linspace(0, 1, 128)),
                       bottom(np.linspace(0, 1, 128))))
cmap = matplotlib.colors.ListedColormap(newcolors, name='OrangeBlue')

# create colormap for legend
cmap_legend = branca.colormap.LinearColormap([cmap(i) for i in np.arange(0,256)], 
                                             vmin=np.nanmin(data_PNW_ti['t'].data), 
                                             vmax=np.nanmax(data_PNW_ti['t'].data), 
                                             caption='Temperature [K]', 
                                             tick_labels=[np.nanmin(data_PNW_ti['t'].data),
                                                          (np.nanmax(data_PNW_ti['t'].data) - np.nanmin(data_PNW_ti['t'].data)) + np.nanmin(data_PNW_ti['t'].data),
                                                          np.nanmax(data_PNW_ti['t'].data)]
                                            )

# "colorize" the data
# from: https://www.linkedin.com/pulse/visualize-dem-interactive-map-chonghua-yin/?trk=related_artice_Visualize%20DEM%20in%20An%20Interactive%20Map_article-card_title
def colorize(array, cmap=cmap):
    normed_data = (array - array.min()) / (array.max() - array.min())    
    cm = cmap    
    return cm(normed_data)
data_colorized = colorize(data_PNW_ti['t'].data)

# add image to map
fm.raster_layers.ImageOverlay(image=data_colorized, 
                              bounds=[[ymin, xmin], [ymax, xmax]], 
                              opacity=0.8,
                              origin='upper', 
                            ).add_to(m)
# add colormap legend to map
m.add_child(cmap_legend)
m

__NOTE:__ This Folium map can be added to the Panel dashboard using the following command (as in [this example](https://panel.holoviz.org/gallery/external/Folium.html)):

`import panel as pn`

`pn.panel(map, height=400)`

In [12]:
import datetime as dt

In [13]:
# Make a slider
date_slider = pn.widgets.DateSlider(name='Date Slider', 
                                    start = data_PNW.valid_time.data[0], 
                                    end = data_PNW.valid_time.data[-1], 
                                    value = data_PNW.valid_time.data[0])


date_slider

DateSlider(end=numpy.datetime64('2018-11-..., name='Date Slider', start=numpy.datetime64('2018-10-..., value=numpy.datetime64('2018-10-...)